In [8]:
import pandas as pd
import re

In [2]:
# Path to the Excel file
file_path = 'data/USGS/Mineral_yearbook/usgs_mineral_yearbook_2020.xlsx'

In [9]:
def clean_usgs(file_path):
    """
    Cleans and structures USGS data tables in an Excel file.
    
    Args:
        file_path (str): Path to the Excel file containing the USGS data.

    Returns:
        dict: A dictionary with each sheet name as keys and cleaned DataFrames as values.
    """
    # Load the file
    excel_data = pd.ExcelFile(file_path)
    
    # Dictionary to store the cleaned DataFrames
    cleaned_dfs = {}
    
    # Iterate over each sheet except the "INFO" sheet
    for sheet_name in excel_data.sheet_names:
        if sheet_name == 'INFO':
            continue  # Skip the INFO sheet as it likely contains metadata
        
        # Load the sheet
        df = excel_data.parse(sheet_name)
        
        # Rename the first column to "Country" for consistency
        df.columns.values[0] = 'Country'
        
        # Remove any numeric references (endnotes) from the country names
        df['Country'] = df['Country'].apply(lambda x: re.sub(r'\d+', '', str(x)).strip())
        
        # Drop rows without any numerical data (removes extraneous text at the end)
        df = df.dropna(subset=df.columns[1:], how='all').reset_index(drop=True)
        
        # Handle multi-line rows where values are on separate lines (if any)
        # We'll check if there's a numeric entry in columns; if not, assume it's a continuation line
        for i in range(1, len(df)):
            if df.iloc[i, 1:].isna().all():  # If the row has no numeric data, assume it's a continuation
                df.iloc[i-1, 0] = f"{df.iloc[i-1, 0]}, {df.iloc[i, 0]}"  # Append to previous row's country name
        
        # Drop any fully empty rows created during merging
        df = df[~df.iloc[:, 1:].isna().all(axis=1)]
        
        # Store the cleaned DataFrame with the sheet name prefix
        cleaned_dfs[f'df_{sheet_name}'] = df.reset_index(drop=True)
    
    return cleaned_dfs


In [10]:
# Apply the cleaning function to the given file and extract 'cobalt_mine' for inspection
cleaned_data = clean_usgs(file_path)
cleaned_data['df_cobalt_mine'].head(20)  # Display the cleaned cobalt mine data to verify the function's output

,Country,2016,2017,2018,2019,2020
0,Australia,5140.0,5034,4878,5746,5632
1,Botswana,248.0,--,--,--,--
2,Brazil,852.0,185,--,30,160
3,Canada,4216.0,3704,3279,3956,3693
4,China,2300.0,2500,2000,2241,2200
5,"Congo (Kinshasa)e,",68000.0,80000,104000,107000,98000
6,"Cubae,",3900.0,3900,3500,3800,3800
7,Finland,690.0,1000,1377,1454,1559
8,"Indonesiae,",1200.0,1200,1200,1100,1100
9,"Madagascare,",3800.0,3600,3300,3400,850


In [12]:
cleaned_data['df_lithium']

,Country,2016,Unnamed: 2,Unnamed: 3,2017,Unnamed: 5,Unnamed: 6,2018,Unnamed: 8,Unnamed: 9,2019,Unnamed: 11,Unnamed: 12,2020,Unnamed: 14,Unnamed: 15
0,Country or locality,Gross weight,Lithium content,LCE3,Gross weight,Lithium content,LCE3,Gross weight,Lithium content,LCE3,Gross weight,Lithium content,LCE3,Gross weight,Lithium content,LCE3
1,Lithium carbonate,24409,4588.892,24409,26559,4993.092,26559,29707,5584.916,29707,29994,5638.872,29994,26911,5059.268,26911
2,Lithium chloride,6468,1054.284,5611.953732,4501,733.663,3905.288149,5005,815.815,4342.583245,4284,698.292,3717.008316,4836,788.268,4195.950564
3,"Australia, spodumene",522181,14537.51904,77383.21385,1706618,47512.24512,252907.680774,1965910,54730.9344,291332.763811,1587980,44209.3632,235326.440314,1427380,39738.2592,211526.753722
4,"Brazil, concentrate",8804,245.10336,1304.685185,10547,293.62848,1562.984399,41000,1141.44,6075.88512,38500,1071.84,5705.40432,51000,1419.84,7557.80832
5,"Canada, spodumene",--,--,--,--,--,--,114000,2433.216,12952.008768,9000,192.096,1022.527008,--,--,--
6,Lithium carbonate,70831,13316.228,70831,73563,13829.844,73563,87029,16361.452,87029,100787,18947.956,100787,114260,21480.88,114260
7,Lithium chloride,1775,289.325,1540.076975,2535,413.205,2199.490215,3826,623.638,3319.625074,1886,307.418,1636.386014,--,--,--
8,Lithium hydroxide,5576,920.04,4897.37292,5280,871.2,4637.3976,6468,1067.22,5680.81206,9934,1639.11,8724.98253,9030,1489.95,7931.00385
9,"China, lithium carbonate equivalent",25400,4775.2,25400,37300,7012.4,37300,37800,7106.4,37800,57500,10810,57500,70600,13272.8,70600
